### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

In [34]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/kanye.txt', sep='/n', header=None)
df = pd.concat([df, pd.read_csv('songs_dataset/Kanye_West.txt', sep='/n', header=None)])
df = pd.concat([df, pd.read_csv('songs_dataset/kanye-west.txt', sep='/n', header=None)])
df.head()

/var/folders/ml/gmqmyfhd39bc7mggbpwznw0r0000gn/T/ipykernel_24433/2070319298.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.

/var/folders/ml/gmqmyfhd39bc7mggbpwznw0r0000gn/T/ipykernel_24433/2070319298.py:3: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.

/var/folders/ml/gmqmyfhd39bc7mggbpwznw0r0000gn/T/ipykernel_24433/2070319298.py:4: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,N-now th-that that don't kill me
1,Can only make me stronger
2,I need you to hurry up now
3,Cause I can't wait much longer
4,I know I got to be right now


Vamos a usar el dataset de canciones de Kanye West

In [35]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 16170


### 1 - Preprocesamiento

In [36]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [37]:
# Demos un vistazo
sentence_tokens[:2]

[['n', 'now', 'th', 'that', 'that', "don't", 'kill', 'me'],
 ['can', 'only', 'make', 'me', 'stronger']]

### 2 - Crear los vectores (word2vec)

In [38]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [39]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [45]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [46]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 16170


In [47]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2303


### 3 - Entrenar embeddings

In [48]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 754229.3125
Loss after epoch 1: 610867.1875
Loss after epoch 2: 555751.375
Loss after epoch 3: 509525.375
Loss after epoch 4: 490145.5
Loss after epoch 5: 478287.5
Loss after epoch 6: 472097.0
Loss after epoch 7: 457186.25
Loss after epoch 8: 432811.5
Loss after epoch 9: 428484.5
Loss after epoch 10: 424174.5
Loss after epoch 11: 422687.0
Loss after epoch 12: 419850.5
Loss after epoch 13: 418317.0
Loss after epoch 14: 417619.5
Loss after epoch 15: 415033.0
Loss after epoch 16: 414518.5
Loss after epoch 17: 401306.5
Loss after epoch 18: 380295.0
Loss after epoch 19: 380038.0
Loss after epoch 20: 376738.0
Loss after epoch 21: 376163.0
Loss after epoch 22: 373554.0
Loss after epoch 23: 373226.0
Loss after epoch 24: 372293.0
Loss after epoch 25: 370157.0
Loss after epoch 26: 368527.0
Loss after epoch 27: 367685.0
Loss after epoch 28: 365853.0
Loss after epoch 29: 365298.0
Loss after epoch 30: 363608.0
Loss after epoch 31: 362783.0
Loss after epoch 32: 362758.0
Loss afte

(4362701, 6662900)

### 4 - Ensayar

In [49]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('bike', 0.3482590317726135),
 ('blame', 0.3426142930984497),
 ('wronger', 0.3407529890537262),
 ('seem', 0.3330535292625427),
 ('lockdown', 0.33089980483055115),
 ('peoples', 0.3243332803249359),
 ('recall', 0.3172752857208252),
 ('dudes', 0.3134351372718811),
 ('judge', 0.3094424903392792),
 ('couture', 0.30928218364715576)]

Con esto podemos ver que palabras como love se relacionan con blame, wronger, lockdown, etc. Todas palabras que en el contexto de las canciones de Kanye West tienen una connotación negativa.

In [62]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('u', -0.005728171672672033),
 ('hotel', -0.008979514241218567),
 ('copped', -0.011036550626158714),
 ('ones', -0.03288609907031059),
 ('label', -0.033720895648002625),
 ('women', -0.04091444984078407),
 ('grind', -0.04176895692944527),
 ('plate', -0.04758397862315178),
 ('children', -0.04801640659570694),
 ('summertime', -0.05179606378078461)]

Algo que podemos sacar de esta comparación de la palabra "love" con las palabras menos similares, es que aparece la palabra "women", por lo que podemos inferir que casi nunca se habla de amor en el contexto de las mujeres en las canciones de Kanye West.

In [63]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["stronger"], topn=10)

[('n', 0.5179175138473511),
 ('fail', 0.40852048993110657),
 ('faster', 0.39622068405151367),
 ('harder', 0.3953496515750885),
 ('anybody', 0.39364510774612427),
 ('th', 0.39095115661621094),
 ('mayonnaise', 0.37812143564224243),
 ('ma', 0.3777454197406769),
 ('run', 0.3686571717262268),
 ('knees', 0.36519381403923035)]

In [64]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["need"], topn=5)

[('joke', 0.3136986494064331),
 ('paradigm', 0.30123090744018555),
 ('ello', 0.30081257224082947),
 ('put', 0.2992168664932251),
 ('flashing', 0.2971089780330658)]

In [67]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["police"])

[('satisfied', 0.026746604591608047),
 ('met', -0.009308302775025368),
 ('cousins', -0.017366500571370125),
 ('any', -0.01783943735063076),
 ('baddest', -0.023940591141581535),
 ('trips', -0.027956059202551842),
 ('had', -0.03290630504488945),
 ('atlanta', -0.035806212574243546),
 ('other', -0.03853766992688179),
 ('theme', -0.041522979736328125)]

In [68]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("dream")
print(vector_love)

[-0.31212252 -0.11257433  0.7584697   0.10845624  0.35580492  0.71279186
 -0.09605224  0.6047854  -0.22640601 -0.2909778   0.3572329  -0.72795266
 -0.00695828  0.21321659 -0.48813272 -0.06538494  0.20967233 -0.39311755
  0.7977855   0.3295214  -0.22386101 -0.07906915 -0.11005306  1.0903326
 -0.09763583 -0.11994542  0.05128441 -0.43061754 -0.15013023 -0.35233253
 -1.108348    0.58065027 -0.2693319  -0.07097553 -0.26919138  0.01157808
 -0.39763308  0.35213825 -0.7743042   0.4324203   0.13418308  0.8106665
 -0.55857     0.243782    0.19409558  0.1119362  -0.30440572  0.8455403
  0.42485243  0.704425    0.25199723 -0.19773112 -0.04633939 -0.717621
  0.53777254  0.3126517   0.86628544  0.29579952 -0.04591222  0.05310382
  0.64539033 -0.17120045  0.2805235  -0.26253402  0.22329943 -0.31737912
  0.50185555  0.62668496 -0.2792441   0.8492668  -0.02900762  0.43942192
  0.32180718 -0.5005264  -0.14574918  0.43660337  0.4852426  -0.20742235
 -0.16941598  0.22811897 -0.15815803  0.36682537 -0.3404

In [69]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('dream', 1.0),
 ('hollerin', 0.3888697028160095),
 ('message', 0.3483738601207733),
 ('france', 0.34741002321243286),
 ('chillin', 0.3420707583427429),
 ('mali', 0.34016308188438416),
 ('selfish', 0.337076872587204),
 ('jordans', 0.33550071716308594),
 ('depends', 0.33533385396003723),
 ('happens', 0.32865288853645325)]

In [70]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('bike', 0.3482590317726135),
 ('blame', 0.3426142930984497),
 ('wronger', 0.3407529890537262),
 ('seem', 0.3330535292625427),
 ('lockdown', 0.33089980483055115),
 ('peoples', 0.3243332803249359),
 ('recall', 0.3172752857208252),
 ('dudes', 0.3134351372718811),
 ('judge', 0.3094424903392792),
 ('couture', 0.30928218364715576)]

### 5 - Visualizar agrupación de vectores

In [71]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [72]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [60]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [ ]:
# Probemos diferentes palabras para analizar similitudes y diferencias en el espacio de embeddings

print("=== Análisis de similitudes y diferencias ===\n")

# 1. Palabras relacionadas con emociones
print("1. Emociones - Palabras similares a 'pain':")
try:
    similar_pain = w2v_model.wv.most_similar(positive=["pain"], topn=5)
    for word, similarity in similar_pain:
        print(f"   {word}: {similarity:.3f}")
except:
    print("   'pain' no está en el vocabulario")

print("\n2. Emociones - Palabras similares a 'light':")
try:
    similar_light = w2v_model.wv.most_similar(positive=["light"], topn=5)
    for word, similarity in similar_light:
        print(f"   {word}: {similarity:.3f}")
except:
    print("   'light' no está en el vocabulario")

# 3. Comparación de similitudes entre conceptos opuestos
print("\n3. Conceptos opuestos - 'high' vs 'low':")
try:
    print("Similares a 'high':")
    similar_high = w2v_model.wv.most_similar(positive=["high"], topn=3)
    for word, similarity in similar_high:
        print(f"   {word}: {similarity:.3f}")
except:
    print("   'high' no está en el vocabulario")

try:
    print("Similares a 'low':")
    similar_low = w2v_model.wv.most_similar(positive=["low"], topn=3)
    for word, similarity in similar_low:
        print(f"   {word}: {similarity:.3f}")
except:
    print("   'low' no está en el vocabulario")

# 4. Cálculo de similitud entre palabras específicas
print("\n4. Similitud directa entre pares de palabras:")
word_pairs = [("love", "heart"), ("time", "moment"), ("eyes", "see"), ("mind", "think")]

for word1, word2 in word_pairs:
    try:
        similarity = w2v_model.wv.similarity(word1, word2)
        print(f"   '{word1}' y '{word2}': {similarity:.3f}")
    except:
        print(f"   Una de las palabras '{word1}' o '{word2}' no está en el vocabulario")

# 5. Analogías usando operaciones vectoriales
print("\n5. Analogías vectoriales:")
print("Intentando: 'you' - 'me' + 'i' = ?")
try:
    analogy = w2v_model.wv.most_similar(positive=["you", "i"], negative=["me"], topn=3)
    for word, similarity in analogy:
        print(f"   {word}: {similarity:.3f}")
except:
    print("   No se pudo calcular la analogía (palabras faltantes)")

=== Análisis de similitudes y diferencias ===

1. Emociones - Palabras similares a 'pain':
   women: 0.350
   paper: 0.343
   deeper: 0.327
   weave: 0.324
   halle: 0.323

2. Emociones - Palabras similares a 'light':
   turbo: 0.374
   sun: 0.355
   theyll: 0.332
   cheating: 0.326
   woo: 0.320

3. Conceptos opuestos - 'high' vs 'low':
Similares a 'high':
   heels: 0.363
   low: 0.351
   explain: 0.341
Similares a 'low':
   eye: 0.398
   dwell: 0.386
   writing: 0.372

4. Similitud directa entre pares de palabras:
   'love' y 'heart': 0.155
   'time' y 'moment': 0.259
   'eyes' y 'see': 0.232
   'mind' y 'think': 0.197

5. Analogías vectoriales:
Intentando: 'you' - 'me' + 'i' = ?
   twista: 0.285
   unfollow: 0.249
   clear: 0.240


Del análisis anterior podemos ver que en las palabras similares a "pain" se encuentran las palabras "deep" y "women", dando a entender que en las canciones de Kanye West el dolor está asociado a las mujeres y a algo profundo.

En cuanto a los conceptos opuestos "high" y "low", podemos ver que las palabras similares a "high" son "heels" (nuevamente relacionado a las mujeres) y la palabra "low" está relacionada con "writing", lo que puede interpretarse como que muchas veces cuando Kanye West se siente "bajo" lo expresa a través de la escritura.

También podemos concluír que no tiene mucho sentido hacer analogías vectoriales con este tipo de embeddings, ya que el contexto es muy específico y no se pueden extrapolar las relaciones entre palabras a otros contextos.